In [484]:
import numpy as np 
import pandas as pd

In [485]:
from operator import index

CT_cond = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\ds_annex_study.csv")
ds = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\ds_CPA_study.csv")
all_data = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\CPA imaging data fields_anonuptodate.csv")

In [486]:
all_data_unique = all_data.drop_duplicates(subset='ID')

ds.insert(1, 'CT-SCAN', ds.ID.str[-1])
ds.ID = ds.ID.str[:-2]
ds.ID[ds.ID == 'RBH CPA 13/'] = ds.ID.str[:-1]


#Merge from all values - demographic 
ds = ds.merge(all_data_unique[['sex', 'ID', 'DOB', 'ETHNIC', 'death_date']],on='ID',how="left")
ds.DOB = ds.DOB.str[:-6]
ds = ds.merge(CT_cond[['ID','COPD','ILD','NTM']],on='ID',how="left")


In [487]:
ds.age = ((pd.to_datetime(ds.date)) - (pd.to_datetime(ds.DOB))) / np.timedelta64(1, 'Y')
ds = ds.drop(columns = 'DOB')
print(ds)

             ID CT-SCAN        date sex ETHNIC death_date COPD ILD  NTM
0     RBH CPA 1       1  16/09/2016   F    NaN        NaN   No  No   No
1     RBH CPA 1       2  16/09/2016   F    NaN        NaN   No  No   No
2     RBH CPA 3       2  07/10/2015   F      A        NaN  Yes  No   No
3     RBH CPA 3       3  25/11/2016   F      A        NaN  Yes  No   No
4     RBH CPA 3       5  15/03/2017   F      A        NaN  Yes  No   No
..          ...     ...         ...  ..    ...        ...  ...  ..  ...
228  RBH CPA 94       1  05/02/2015   F      B  15-Apr-16   No  No   No
229  RBH CPA 96       1  23/11/2016   F      J        NaN   No  No   No
230  RBH CPA 97       1  13/03/2013   F      Z        NaN   No  No   No
231  RBH CPA 98       1  19/07/2017   M      A  27-Jun-18  Yes  No  Yes
232  RBH CPA 99       1  20/11/2017   M      A        NaN  Yes  No   No

[233 rows x 9 columns]


C:\Users\USER\AppData\Local\Temp\ipykernel_21756\3383720863.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ds.age = ((pd.to_datetime(ds.date)) - (pd.to_datetime(ds.DOB))) / np.timedelta64(1, 'Y')


In [488]:
#Defining patients statuts after 5 years - 0: alive, 1: dead, nan: unknown
ds.fy = pd.to_datetime(pd.to_datetime(ds.date) + pd.offsets.DateOffset(years=5))
ds['death5'] = 0
ds['death5'][ds.fy  > ds.death_date] = 1
ds['death5'][ds.fy  > pd.to_datetime('01-05-2021')] =  np.nan



C:\Users\USER\AppData\Local\Temp\ipykernel_21756\187792707.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ds.fy = pd.to_datetime(pd.to_datetime(ds.date) + pd.offsets.DateOffset(years=5))
C:\Users\USER\AppData\Local\Temp\ipykernel_21756\187792707.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['death5'][ds.fy  > ds.death_date] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_21756\187792707.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['death5'][ds.fy  > pd.to_datetime('01-05-2021')] 

In [489]:
#Collecting variables closest to the CT scan for each patient 
all_data.sampleCollectDateTime = pd.to_datetime(all_data.sampleCollectDateTime)
intermediate = pd.DataFrame()
intermediate['ID'] = ds['ID']
intermediate['date'] = ds['date']
intermediate['closest'] = 0

def closer_date(set_of_dates, goal):
    return min(set_of_dates, key=lambda x: abs(x - goal))



#find the best observation date for each 'patient'
#(1) find the date of observation for each patient 
date_set = []
for index, row in ds.iterrows():
    date_set = np.array(all_data[all_data.ID == row.ID].sampleCollectDateTime.values)
    date = pd.to_datetime(row.date , dayfirst=True)
    if np.isnan(date_set).all():
        closest_date = np.nan
    else:
        closest_date = closer_date(date_set, date)
        print(closest_date)
    intermediate.loc[index,'closest']= closest_date



2016-09-13T00:00:00.000000000
2016-09-13T00:00:00.000000000
2017-02-21T00:00:00.000000000
2017-02-21T00:00:00.000000000
2017-02-21T00:00:00.000000000
2012-09-13T00:00:00.000000000
2014-01-23T00:00:00.000000000
2014-09-18T00:00:00.000000000
2014-09-18T00:00:00.000000000
2016-09-14T00:00:00.000000000
2016-09-14T00:00:00.000000000
2016-09-14T00:00:00.000000000
2009-03-12T00:00:00.000000000
2012-12-13T00:00:00.000000000
2018-05-12T00:00:00.000000000
2018-05-12T00:00:00.000000000
2018-05-12T00:00:00.000000000
2010-04-29T00:00:00.000000000
2011-03-25T00:00:00.000000000
2012-06-18T00:00:00.000000000
2013-03-13T00:00:00.000000000
2014-09-15T00:00:00.000000000
2016-03-05T00:00:00.000000000
2017-01-03T00:00:00.000000000
2017-10-14T00:00:00.000000000
2010-02-15T00:00:00.000000000
2013-07-11T00:00:00.000000000
2014-01-13T00:00:00.000000000
2012-05-07T00:00:00.000000000
2013-11-19T00:00:00.000000000
2015-04-17T00:00:00.000000000
2017-08-12T00:00:00.000000000
2018-01-18T00:00:00.000000000
2016-02-22

In [496]:
print(intermediate)
ds['crp'] = 0
for index, row in ds.iterrows():
    date = intermediate.loc[index, 'closest']
    crp_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].crp.values
    ds.loc[index, 'crp'] = crp_val

ds.crp

             ID        date                        closest
0     RBH CPA 1  16/09/2016  2016-09-13T00:00:00.000000000
1     RBH CPA 1  16/09/2016  2016-09-13T00:00:00.000000000
2     RBH CPA 3  07/10/2015  2017-02-21T00:00:00.000000000
3     RBH CPA 3  25/11/2016  2017-02-21T00:00:00.000000000
4     RBH CPA 3  15/03/2017  2017-02-21T00:00:00.000000000
..          ...         ...                            ...
228  RBH CPA 94  05/02/2015  2015-03-23T00:00:00.000000000
229  RBH CPA 96  23/11/2016  2016-11-23T00:00:00.000000000
230  RBH CPA 97  13/03/2013  2012-11-21T00:00:00.000000000
231  RBH CPA 98  19/07/2017  2017-08-24T00:00:00.000000000
232  RBH CPA 99  20/11/2017  2017-11-20T00:00:00.000000000

[233 rows x 3 columns]


C:\Users\USER\AppData\Local\Temp\ipykernel_21756\431943575.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  crp_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].crp.values


ValueError: Must have equal len keys and value when setting with an iterable